# Train behavior cloning

Train a behavior cloning based robot controller. 
* Code for loading and pre-processing the training data, typically from a set of demonstrations as specified in an exp/run
* Train the controller. 
* The trained controllers should be saved into the exp/run

In [1]:
import sys
sys.path.append("..")

from exp_run_config import Config
Config.PROJECTNAME = "BerryPicker"

import pathlib
from tqdm import tqdm
import pprint
import torch
torch.manual_seed(1)

from bc_trainingdata import create_trainingdata_bc
from bc_factory import create_bc_model
from bc_LSTM_MDN import mdn_loss

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"
print(f"Using device: {device}")

***ExpRun**: Loading pointer config file:
	/Users/lboloni/.config/BerryPicker/mainsettings.yaml
***ExpRun**: Loading machine-specific config file:
	/Users/lboloni/Google Drive/My Drive/LotziStudy/Code/PackageTracking/BerryPicker/settings/settings-szenes.yaml
Using device: cpu


### Exp/run initialization
Create the exp/run-s that describe the parameters of the training. Some of the code here is structured in such a way as to make the notebook automatizable with papermill.

In [ ]:
# *** Initialize the variables with default values 
# *** This cell should be tagged as parameters     
# *** If papermill is used, some of the values will be overwritten 

# If it is set to true, the exprun will be recreated from scratch
creation_style = "exist-ok"

# If not None, set an external experiment path
external_path = None
# If not None, set an output path
data_path = None

experiment = "behavior_cloning"
# run = "bc_mlp_00"
# run = "bc_lstm_00"
run = "bc_lstm_resid_00"
# run = "bc_lstm_mdn_00"
# exp = Config().get_experiment(experiment, run)


***ExpRun**: Configuration for exp/run: behavior_cloning/bc_lstm_resid_00 successfully loaded
Experiment:
    batch_size: 64
    control_size: 6
    controller: bc_LSTM_Residual
    controller_file: controller.pth
    data_dir: /Users/lboloni/Documents/Develop/Data/BerryPicker/behavior_cloning/bc_lstm_resid_00
    epochs: 10
    exp_robot: robot_al5d
    exp_run_sys_indep_file: /Users/lboloni/Documents/Develop/Github/BerryPicker/BerryPicker/src/experiment_configs/behavior_cloning/bc_lstm_resid_00.yaml
    exp_sp: sensorprocessing_conv_vae
    experiment_name: behavior_cloning
    hidden_size: 32
    loss: MSELoss
    name: bc_lstm_resid_00
    optimizer: Adam
    optimizer_lr: 0.001
    run_name: bc_lstm_resid_00
    run_robot: position_controller_00
    run_sp: sp_vae_256
    sequence_lenght: 10
    sequence_length: 10
    subrun_name: null
    time_started: '2025-06-29 14:35:20.532518'
    training_data:
    - - random-both-cameras-video
      - '2025_03_08__14_15_53'
      - dev2
  

In [ ]:
if external_path:
    external_path = pathlib.Path(external_path)
    assert external_path.exists()
    Config().set_experiment_path(external_path)
    Config().copy_experiment("sensorprocessing_conv_vae")
    Config().copy_experiment("robot_al5d")
    Config().copy_experiment("demonstration")
    Config().copy_experiment("behavior_cloning")
if data_path:
    data_path = pathlib.Path(data_path)
    assert data_path.exists()
    Config().set_experiment_data(data_path)

exp = Config().get_experiment(experiment, run, creation_style=creation_style)
pprint.pprint(exp)
exp_sp = Config().get_experiment(exp["exp_sp"], exp["run_sp"])
exp_robot = Config().get_experiment(exp["exp_robot"], exp["run_robot"])

### Training an RNN model
Functions for training an RNN type model. These models assume that the input is a sequence $[z_{t-k},...z_{t}]$ while the output is the next action $a_{t+1}$

In [3]:
def validate_bc_rnn(model, validation_loss, data, device):
    """Calculates the average validation error for the behavior cloning model using an RNN with the specific criterion function. Uses the z_validation an a_validation fields in "data". The inputs and the targets a list of individual input and target. 
    CHECK: I think that the target is supposed to be the last output of the RNN when the whole input string had been passed through it. 
    The model is reset before each of the strings (i.e. state is not transferred)
    model: an LSTM or similar model that can consume a sequence of inputs
    criterion: any function that calculates the distance between the targets
    """
    num_sequences = data["z_validation"].shape[0]
    model.eval()
    val_loss = 0
    with torch.no_grad():  # Disable gradient computation
        for i in range(num_sequences):
            input_seq = data["z_validation"][i].to(device)
            target = data["a_validation"][i].to(device)
            # Reshape for batch compatibility
            input_seq = input_seq.unsqueeze(0)  # Shape: [1, sequence_length, latent_size]
            target = target.unsqueeze(0)        # Shape: [1, latent_size]
            if not model.stochastic:
                outputs = model(input_seq)
                loss = validation_loss(outputs, target)
            else: # for MDN, the output is sampling
                outputs = model.forward_and_sample(input_seq)
            loss = validation_loss(outputs, target)
            val_loss += loss.item()
    avg_loss = val_loss / num_sequences
    return avg_loss


In [4]:
def train_bc_rnn(model, optimizer, criterion, data, num_epochs, batch_size=32):
    """Train a behavior cloning model using a sequence model (eg. an RNN)
    Uses a writer for TensorBoard _and_ tqdm
    Model with batch
    """

    exp.start_timer("train")
    num_sequences = data["z_train"].shape[0]
    num_batches = num_sequences // batch_size

    for epoch in tqdm(range(num_epochs)):        
        model.train()
        # Loop over each sequence in the batch
        training_loss = 0
        for i in range(num_batches):
            # Prepare input and target
            input_seq = data["z_train"][i * batch_size: (i+1)* batch_size].to(device) # Shape: [batch_size, sequence_length, latent_size]
            target = data["a_train"][i * batch_size: (i+1)* batch_size].to(device) # Shape: [batch_size, latent_size]
            # Forward pass
            output = model(input_seq) # Shape: [batch_size, output_size]
            # Check for MDN, that is different
            if not model.stochastic:
                loss = criterion(output, target)
            else: 
                loss = mdn_loss(target, *output)
            training_loss += loss.item()
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()        
        avg_training_loss = training_loss / num_sequences

        #if writer is not None:
        #    writer.add_scalar("TrainingLoss", avg_training_loss, epoch)
        #    writer.add_scalar("ValidationLoss", avg_validation_loss, epoch)
        #    writer.flush()
        if (epoch+1) % 5 == 0: # was 0
            avg_validation_loss = validate_bc_rnn(model, criterion, data, device)
            print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_training_loss:.4f} Validation Loss: {avg_validation_loss:.4f} ')
    print("Training complete.")
    exp.end_timer("train")


### Train the model 

Creates and trains a behavior cloning model specified by the exp.

In [5]:
model, validation_loss, optimizer = create_bc_model(exp, exp_sp, device)
print(model)

bc_LSTM_Residual(
  (lstm_1): LSTM(256, 32, batch_first=True)
  (lstm_2): LSTM(32, 32, batch_first=True)
  (lstm_3): LSTM(32, 32, batch_first=True)
  (fc): Linear(in_features=32, out_features=6, bias=True)
)


In [ ]:
controller_path = pathlib.Path(exp.data_dir(), exp["controller_file"])

if controller_path.exists():
    print("f***Train_BehaviorCloning controller already exists, exiting.\n{controller_path}")
else:
    data = create_trainingdata_bc(exp, exp_sp, exp_robot, device="cpu")
    # Training Loop
    num_epochs = exp["epochs"]
    batch_size = exp["batch_size"]

    # Create a SummaryWriter instance
    # where does the logdir go???
    # writer = SummaryWriter(logdir="/home/lboloni/runs/example")
    train_bc_rnn(
            model, optimizer, validation_loss, data=data,
            num_epochs=num_epochs, batch_size=batch_size)
    # writer.close()
    torch.save(model.state_dict(), controller_path)


***Timer*** data_preparation started
***ExpRun**: Experiment default config /Users/lboloni/Documents/Develop/Github/BerryPicker/BerryPicker/src/experiment_configs/demonstration/_defaults_demonstration.yaml was empty, ok.
***ExpRun**: Configuration for exp/run: demonstration/random-both-cameras-video successfully loaded
***ExpRun**: Experiment default config /Users/lboloni/Documents/Develop/Github/BerryPicker/BerryPicker/src/experiment_configs/demonstration/_defaults_demonstration.yaml was empty, ok.
***ExpRun**: Configuration for exp/run: demonstration/random-both-cameras-video successfully loaded


/Users/lboloni/Documents/Develop/Github/BerryPicker/BerryPicker/src/behavior_cloning/bc_trainingdata.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs_list.append(torch.tensor(input_seq))
/Users/lboloni/Documents/Develop/Github/BerryPicker/BerryPicker/src/behavior_cloning/bc_trainingdata.py:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets_list.append(torch.tensor(target))


***ExpRun**: Experiment default config /Users/lboloni/Documents/Develop/Github/BerryPicker/BerryPicker/src/experiment_configs/demonstration/_defaults_demonstration.yaml was empty, ok.
***ExpRun**: Configuration for exp/run: demonstration/random-both-cameras-video successfully loaded
***ExpRun**: Experiment default config /Users/lboloni/Documents/Develop/Github/BerryPicker/BerryPicker/src/experiment_configs/demonstration/_defaults_demonstration.yaml was empty, ok.
***ExpRun**: Configuration for exp/run: demonstration/random-both-cameras-video successfully loaded
***Timer*** data_preparation finished in 5.120951 seconds
***Timer*** train started


 80%|████████  | 8/10 [00:00<00:00, 11.29it/s]

Epoch [5/10], Training Loss: 0.0370 Validation Loss: 2.4901 


100%|██████████| 10/10 [00:01<00:00,  9.90it/s]

Epoch [10/10], Training Loss: 0.0260 Validation Loss: 1.8220 
Training complete.
***Timer*** train finished in 1.026546 seconds
